In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

from umap import UMAP

from get_metrics import get_metrics_classification, get_metrics_regression

import warnings
warnings.filterwarnings("ignore");

PATH = '../MTS ML Cup/'
RAND = 42

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

Прочтем файл с данными

In [2]:
data = pd.read_csv(PATH+'data_cleaned.csv')
data.head()

,Unnamed: 0,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,age,is_male
0,0,Краснодарский край,Геленджик,Apple,iPhone X,instagram.com,iOS,67590.0,2021-06-29,day,2,214519,29.0,0
1,1,Краснодарский край,Геленджик,Apple,iPhone X,instagram.com,iOS,67590.0,2021-06-29,evening,6,214519,29.0,0
2,2,Краснодарский край,Геленджик,Apple,iPhone X,googleads.g.doubleclick.net,iOS,76665.0,2021-07-19,day,3,60193,31.0,1
3,3,Краснодарский край,Геленджик,Apple,iPhone X,vk.com,iOS,73374.0,2021-07-28,day,1,319852,28.0,0
4,4,Краснодарский край,Геленджик,Apple,iPhone X,tpc.googlesyndication.com,iOS,73374.0,2021-07-24,morning,1,319852,28.0,0


В процессе сохранения таблицы в csv-файл и переноса таблицы из jupyter ноутбука с EDA в этот появилась лишняя колонка ***Unnamed: 0*** 

In [3]:
data.drop('Unnamed: 0', axis=1, inplace=True)

Посмотрим на количество пропусков 

In [4]:
data.isna().sum()

region_name                0
city_name                  0
cpe_manufacturer_name      0
cpe_model_name             0
url_host                 145
cpe_model_os_type          0
price                      0
date                       0
part_of_day                0
request_cnt                0
user_id                    0
age                        0
is_male                    0
dtype: int64

В jupyter ноутбуке с EDA в данной таблице в ***url_host*** присутствовали значения "null" (как раз в количестве 145) и не идентифицировались как пропуски. А в процессе переноса данных из одного ноутбука в другой "null" трансформировался в NaN.  

In [5]:
data.dropna(inplace=True)

Посмотрим на типы данных столбцов

In [6]:
data.dtypes

region_name               object
city_name                 object
cpe_manufacturer_name     object
cpe_model_name            object
url_host                  object
cpe_model_os_type         object
price                    float64
date                      object
part_of_day               object
request_cnt                int64
user_id                    int64
age                      float64
is_male                    int64
dtype: object

Необходимо перевести тип данных столбца ***date*** в datetime, а также object столбцы в category

In [7]:
data.date = data.date.astype("datetime64")

cat_cols = data.select_dtypes('object').columns
data[cat_cols] = data[cat_cols].astype("category")

Теперь нужно закодировать категориальные признаки. Для ***part_of_day*** зададим лейблы для каждого значения (тк в этом признаке присутствует градация), а колонку с датой разобьем на три составляющие: год, месяц, день. Остальные категориальные признаки закодируем с помощью OHE и позже создадим для них эмбеддинги

In [8]:
def part_of_day_labels(part_of_day):
    if part_of_day == 'morning':
        return 1
    elif part_of_day == 'day':
        return 2
    elif part_of_day == 'evening':
        return 3
    elif part_of_day == 'night':
        return 4
        
data['part_of_day'] = data.part_of_day.apply(part_of_day_labels)
data.part_of_day = data.part_of_day.astype("int64")  #что бы при OHE для лейблов не создавались отдельные колонки

In [9]:
data_ohe = pd.get_dummies(data)

In [10]:
data_ohe['year'] = data_ohe.date.dt.year
data_ohe['month'] = data_ohe.date.dt.month
data_ohe['day'] = data_ohe.date.dt.day

data_ohe.drop('date', axis=1, inplace=True)

# Классификация. Определение пола

Теперь разобьем данные на обучающую и тестовую выборки. Также, создадим эмбеддинги для закодированных OHE признаков.

In [11]:
X = data_ohe.drop(['is_male', 'age', 'user_id'], axis=1)
y = data_ohe[['is_male', 'age']]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=RAND,
                                                    stratify=y.is_male)

In [12]:
%%time
um = UMAP(n_components=100,random_state=RAND)

X_embeddings_train = um.fit_transform(X_train.drop(['price', 'part_of_day', 'request_cnt',
                                                    'year', 'month', 'day'], axis=1))
X_embeddings_test = um.transform(X_test.drop(['price', 'part_of_day', 'request_cnt',
                                              'year', 'month', 'day'], axis=1))

CPU times: user 40min 19s, sys: 20min 28s, total: 1h 48s
Wall time: 29min 3s


Объединим эмбеддинги в таблицу с остальными признаками

In [13]:
X_embeddings_train = pd.DataFrame(X_embeddings_train)
X_train = X_train[['price', 'part_of_day', 'request_cnt',
                   'year', 'month', 'day']].reset_index()
X_embeddings_train['index'] = X_train['index']
X_train = X_train.merge(X_embeddings_train, on='index')
X_train.drop('index', axis=1, inplace=True)

X_embeddings_test = pd.DataFrame(X_embeddings_test)
X_test = X_test[['price', 'part_of_day', 'request_cnt',
                 'year', 'month', 'day']].reset_index()
X_embeddings_test['index'] = X_test['index']
X_test = X_test.merge(X_embeddings_test, on='index')
X_test.drop('index', axis=1, inplace=True)

Дополнительно разобьем данные на тренировочную и валидационную выборки

In [14]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND,
                                                    stratify=y_train.is_male)

Размерности выборок

In [15]:
print(f'X_train: {X_train.shape}\nX_train_: {X_train_.shape}\n' \
      f'X_val: {X_val.shape}\nX_test: {X_test.shape}')

X_train: (138237, 106)
X_train_: (116119, 106)
X_val: (22118, 106)
X_test: (34560, 106)


Теперь можно приступать к моделированию

## Random Forest

Функция для проверки на переобучение (для классификации)

In [16]:
def check_overfitting(model, X_train, y_train, X_test, y_test):
    """
    Проверка на overfitting для классификации 
    """
    y_score_train = model.predict_proba(X_train)
    y_score_test = model.predict_proba(X_test)
    value_train = roc_auc_score(y_train, y_score_train[:, 1])
    value_test = roc_auc_score(y_test, y_score_test[:, 1])

    print(f'{roc_auc_score.__name__} train: %.3f' % value_train)
    print(f'{roc_auc_score.__name__} test: %.3f' % value_test)
    print(f'delta = {(abs(value_train - value_test)/value_test*100):.1f} %')

Обучим модель

In [17]:
%%time
rf_clf = RandomForestClassifier(class_weight='balanced', random_state=RAND)
rf_clf.fit(X_train, y_train.is_male);

CPU times: user 3min 16s, sys: 1.03 s, total: 3min 17s
Wall time: 3min 18s


RandomForestClassifier(class_weight='balanced', random_state=42)

Проверим факт переобучения

In [18]:
check_overfitting(rf_clf,
                  X_train,
                  y_train.is_male,
                  X_test,
                  y_test.is_male)

roc_auc_score train: 1.000
roc_auc_score test: 0.823
delta = 21.4 %


Сделаем предикт

In [19]:
predict = rf_clf.predict(X_test)
proba = rf_clf.predict_proba(X_test)

Таблица с метриками

In [20]:
metrics = get_metrics_classification(y_test=y_test.is_male,
                                     y_pred=predict,
                                     y_score=proba,
                                     name='BaselineRandomForestClassifier')
metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,BaselineRandomForestClassifier,0.733015,0.823465,0.741892,0.74077,0.74133,0.509302


## LightGBM

Обучим модель

In [21]:
%%time
ratio = float(np.sum(y_train_.is_male == 0)) / np.sum(y_train_.is_male == 1)

lgbm_clf = LGBMClassifier(random_state=RAND, scale_pos_weight=ratio)
eval_set = [(X_val, y_val.is_male)]

lgbm_clf.fit(X_train_,
             y_train_.is_male,
             eval_metric='auc',
             eval_set=eval_set,
             verbose=False,
             early_stopping_rounds=100);

CPU times: user 11.6 s, sys: 936 ms, total: 12.5 s
Wall time: 1.94 s


LGBMClassifier(random_state=42, scale_pos_weight=0.9361879512447268)

Проверим факт переобучения

In [22]:
check_overfitting(lgbm_clf,
                  X_train_,
                  y_train_.is_male,
                  X_test,
                  y_test.is_male)

roc_auc_score train: 0.832
roc_auc_score test: 0.769
delta = 8.2 %


Сделаем предикт

In [23]:
predict = lgbm_clf.predict(X_test)
proba = lgbm_clf.predict_proba(X_test)

Таблица с метриками

In [24]:
metrics = metrics.append(get_metrics_classification(y_test=y_test.is_male,
                                                    y_pred=predict,
                                                    y_score=proba,
                                                    name='BaselineLGBMClassifier'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,BaselineRandomForestClassifier,0.733015,0.823465,0.741892,0.740770,0.741330,0.509302
0,BaselineLGBMClassifier,0.690365,0.768939,0.711329,0.673987,0.692155,0.598513


## CatBoost

Обучим модель

In [25]:
%%time
ctbst_clf = CatBoostClassifier(random_state=RAND,
                               scale_pos_weight=ratio,
                               eval_metric='AUC')

ctbst_clf.fit(X_train_,
              y_train_.is_male,
              eval_set=eval_set,
              verbose=False,
              early_stopping_rounds=100);

CPU times: user 1min 24s, sys: 9.49 s, total: 1min 33s
Wall time: 17.4 s


Проверим факт переобучения

In [26]:
check_overfitting(ctbst_clf,
                  X_train_,
                  y_train_.is_male,
                  X_test,
                  y_test.is_male)

roc_auc_score train: 0.931
roc_auc_score test: 0.859
delta = 8.4 %


Сделаем предикт

In [27]:
predict = ctbst_clf.predict(X_test)
proba = ctbst_clf.predict_proba(X_test)

Таблица с метриками

In [28]:
metrics = metrics.append(get_metrics_classification(y_test=y_test.is_male,
                                                    y_pred=predict,
                                                    y_score=proba,
                                                    name='BaselineCatBoostClassifier'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,BaselineRandomForestClassifier,0.733015,0.823465,0.741892,0.740770,0.741330,0.509302
0,BaselineLGBMClassifier,0.690365,0.768939,0.711329,0.673987,0.692155,0.598513
0,BaselineCatBoostClassifier,0.770544,0.858866,0.790488,0.756121,0.772923,0.488796


## Выводы

Учитывая все факторы (время обучения, процент переобучения, значения метрик на тестовой выборке) можно сказать, что лучшие результаты показал CatBoost. Random Forest обучался очень долго в сравнении с бустингами + очень большой процент переобучения. LightGBM показал самую высокую скорость обучения и самый низкий процент переобучения, но при этом на порядок слабее результаты по сравнению с другими моделями. 

# Регрессия. Определение возраста

Функция для проверки на переобучение (для регрессии)

In [29]:
def check_overfitting(model, X_train, y_train, X_test, y_test, metric_fun):
    """
    Проверка на overfitting для регрессии
    """
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    value_train = metric_fun(y_train, y_pred_train)
    value_test = metric_fun(y_test, y_pred_test)

    print(f'{metric_fun.__name__} train: %.3f' % value_train)
    print(f'{metric_fun.__name__} test: %.3f' % value_test)
    print(f'delta = {(abs(value_train - value_test)/value_test*100):.1f} %')

## Random Forest

Обучим модель

In [30]:
%%time
rf_reg = RandomForestRegressor(random_state=RAND)
rf_reg.fit(X_train, y_train.age);

CPU times: user 24min 18s, sys: 2.25 s, total: 24min 20s
Wall time: 31min


RandomForestRegressor(random_state=42)

Проверим факт переобучения

In [31]:
check_overfitting(rf_reg,
                  X_train_,
                  y_train_.age,
                  X_test,
                  y_test.age,
                  metric_fun=mean_absolute_error)

mean_absolute_error train: 1.361
mean_absolute_error test: 4.909
delta = 72.3 %


Сделаем предикт

In [32]:
predict = rf_reg.predict(X_test)

Таблица с метриками

In [33]:
metrics = get_metrics_regression(y_test=y_test.age,
                                 y_pred=predict,
                                 X_test=X_test,
                                 name='BaselineRandomForestRegressor')

round(metrics.set_index('model'), 3)

,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
model,,,,,,,,
BaselineRandomForestRegressor,4.909,52.926,7.275,0.191,0.579,-6.056,14.47,13.385


## LightGBM

Обучим модель

In [34]:
%%time
lgbm_reg = LGBMRegressor(random_state=RAND)
eval_set = [(X_val, y_val.age)]

lgbm_reg.fit(X_train_,
             y_train_.age,
             eval_metric="mae",
             eval_set=eval_set,
             verbose=False,
             early_stopping_rounds=100);

CPU times: user 11.4 s, sys: 936 ms, total: 12.4 s
Wall time: 1.96 s


LGBMRegressor(random_state=42)

Проверим факт переобучения

In [35]:
check_overfitting(lgbm_reg,
                  X_train_,
                  y_train_.age,
                  X_test,
                  y_test.age,
                  metric_fun=mean_absolute_error)

mean_absolute_error train: 7.241
mean_absolute_error test: 7.519
delta = 3.7 %


Сделаем предикт

In [36]:
predict = lgbm_reg.predict(X_test)

Таблица с метриками

In [37]:
metrics = metrics.append(get_metrics_regression(y_test=y_test.age,
                                                y_pred=predict,
                                                X_test=X_test,
                                                name='BaselineLGBMRegressor'))

round(metrics.set_index('model'), 3)

,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
model,,,,,,,,
BaselineRandomForestRegressor,4.909,52.926,7.275,0.191,0.579,-6.056,14.470,13.385
BaselineLGBMRegressor,7.519,90.378,9.507,0.247,0.281,-7.385,21.775,20.501


## CatBoost

Обучим модель

In [38]:
%%time
ctbst_reg = CatBoostRegressor(random_state=RAND, 
                              eval_metric="MAE")

ctbst_reg.fit(X_train_,
              y_train_.age,
              eval_set=eval_set,
              verbose=False,
              early_stopping_rounds=100);

CPU times: user 1min 11s, sys: 5.65 s, total: 1min 17s
Wall time: 13 s


Проверим факт переобучения

In [39]:
check_overfitting(ctbst_reg,
                  X_train_,
                  y_train_.age,
                  X_test,
                  y_test.age,
                  metric_fun=mean_absolute_error)

mean_absolute_error train: 5.982
mean_absolute_error test: 6.613
delta = 9.5 %


Сделаем предикт

In [40]:
predict = ctbst_reg.predict(X_test)

Таблица с метриками

In [41]:
metrics = metrics.append(get_metrics_regression(y_test=y_test.age,
                                                y_pred=predict,
                                                X_test=X_test,
                                                name='BaselineCatBoostRegressor'))

round(metrics.set_index('model'), 3)

,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
model,,,,,,,,
BaselineRandomForestRegressor,4.909,52.926,7.275,0.191,0.579,-6.056,14.470,13.385
BaselineLGBMRegressor,7.519,90.378,9.507,0.247,0.281,-7.385,21.775,20.501
BaselineCatBoostRegressor,6.613,74.136,8.610,0.225,0.410,-6.342,19.138,18.029


## Выводы

Лучшие значения метрик дал Random Forest, но при этом он обучался около получаса + очень высокий процент переобучения по сравнению с бустингами. LightGBM показал результаты хуже, чем Random Forest и CatBoost, но в то же время у него самое быстрое время обучения и самый маленький процент overfitting. Ну а CatBoost стал золотой серединой между двумя моделями.

Сохраним выборки в отдельные csv файлы для дальнейшей работы (Tuning)

In [45]:
X_train.to_csv(PATH+'X_train.csv')
X_train_.to_csv(PATH+'X_train_.csv')
X_val.to_csv(PATH+'X_val.csv')
X_test.to_csv(PATH+'X_test.csv')

y_train.to_csv(PATH+'y_train.csv')
y_train_.to_csv(PATH+'y_train_.csv')
y_val.to_csv(PATH+'y_val.csv')
y_test.to_csv(PATH+'y_test.csv')